In [1]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 导入必要模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template

In [3]:
# 导入必要模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template
import ipywidgets as widgets # type: ignore
from IPython.display import display, clear_output # type: ignore

In [4]:
# def create_model_and_tokenizer():
#     """加载原始模型和tokenizer"""
#     # 4bit量化配置
#     quantization_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_compute_dtype=torch.float16,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type="nf4"
#     )
    
#     # 加载tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(
#         "google/gemma-2-9b",
#         cache_dir="/root/autodl-tmp/gemma",
#         trust_remote_code=True,
#         local_files_only=True
#     )
    
#     # 加载基础模型
#     base_model = AutoModelForCausalLM.from_pretrained(
#         "google/gemma-2-9b",
#         cache_dir="/root/autodl-tmp/gemma",
#         device_map="auto",
#         torch_dtype=torch.float16,
#         quantization_config=quantization_config,
#         local_files_only=True
#     )
    
#     # 加载微调的LoRA权重
#     model = PeftModel.from_pretrained(
#         base_model,
#         "/root/autodl-tmp/models/stage1/checkpoints/gemma-base-zh/checkpoint-20000"  # 微调模型的路径
#     )
    
#     return model, tokenizer

In [5]:
# def chat_with_model():
#     # 加载模型和tokenizer
#     model, tokenizer = create_model_and_tokenizer()
#     model.eval()  # 设置为评估模式
    
#     print("模型已加载完成，可以开始对话了！(输入 'quit' 结束对话)")
    
#     while True:
#         # 获取用户输入
#         user_input = input("\n用户: ")
#         if user_input.lower() == 'quit':
#             break
            
#         # 构建输入格式
#         prompt = f"<start_of_turn>user\n{user_input}\n<end_of_turn><eos>\n<start_of_turn>model\n"
        
#         # 生成回答
#         inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=512,
#             temperature=0.7,
#             top_p=0.95,
#             do_sample=True,
#             repetition_penalty=1.1
#         )
        
#         # 解码模型输出
#         response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
#         # 提取模型的回答部分（去除输入部分）
#         response = response.split("<start_of_turn>model\n")[-1].split("<end_of_turn>")[0].strip()
        
#         print("\n助手:", response)

# if __name__ == "__main__":
#     chat_with_model()

In [6]:
# # 加载模型和tokenizer
# model, tokenizer = create_model_and_tokenizer()
# model.eval()

# # 修改输出区域的样式
# conversation_output = widgets.Output(
#     layout=widgets.Layout(
#         width='100%',  # 设置宽度为100%
#         max_width='1200px',  # 设置最大宽度
#         min_height='400px',  # 设置最小高度
#         border='1px solid #ddd',  # 添加边框
#         overflow='auto',  # 添加滚动条
#         word_wrap='break-word'  # 确保文本自动换行
#     )
# )

# # 修改输入框的样式
# input_box = widgets.Text(
#     value='',
#     placeholder='请输入您的问题...',
#     description='用户:',
#     disabled=False,
#     layout=widgets.Layout(
#         width='80%',  # 调整宽度比例
#         max_width='1000px'  # 设置最大宽度
#     )
# )

# # 修改按钮样式
# send_button = widgets.Button(
#     description='发送',
#     disabled=False,
#     button_style='primary',
#     tooltip='发送消息',
#     icon='paper-plane',
#     layout=widgets.Layout(width='100px')  # 设置固定宽度
# )

# clear_button = widgets.Button(
#     description='清空对话',
#     button_style='warning',
#     tooltip='清空对话历史',
#     layout=widgets.Layout(width='100px')  # 设置固定宽度
# )

# # 修改按钮容器样式
# button_container = widgets.HBox(
#     [send_button, clear_button],
#     layout=widgets.Layout(
#         width='auto',
#         margin='0 0 0 10px'  # 添加左边距
#     )
# )

# # 修改输入区域容器样式
# input_container = widgets.HBox(
#     [input_box, button_container],
#     layout=widgets.Layout(
#         width='100%',
#         max_width='1200px',
#         justify_content='space-between'  # 均匀分布空间
#     )
# )

# # 修改主容器样式
# main_container = widgets.VBox(
#     [conversation_output, input_container],
#     layout=widgets.Layout(
#         width='100%',
#         max_width='1200px',
#         padding='10px',
#         margin='0 auto'  # 居中显示
#     )
# )

# def on_send_button_clicked(b):
#     user_input = input_box.value
#     if not user_input.strip():
#         return
        
#     input_box.value = ''
    
#     with conversation_output:
#         print(f"\n用户: {user_input}\n")
        
#         prompt = f"<start_of_turn>user\n{user_input}\n<end_of_turn><eos>\n<start_of_turn>model\n"
        
#         inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#         outputs = model.generate(
#             **inputs,
#             max_new_tokens=512,
#             temperature=0.7,
#             top_p=0.95,
#             do_sample=True,
#             repetition_penalty=1.1
#         )
        
#         response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         response = response.split("<start_of_turn>model\n")[-1].split("<end_of_turn>")[0].strip()
        
#         # 使用HTML格式化输出
#         display(HTML(f"""
#         <div style="white-space: pre-wrap; word-wrap: break-word; width: 800px;">
#         <b>助手:</b> {response}
#         </div>
#         """))
#         print("-" * 80)

# def on_clear_button_clicked(b):
#     conversation_output.clear_output()

# # 绑定按钮事件
# send_button.on_click(on_send_button_clicked)
# clear_button.on_click(on_clear_button_clicked)

# # 设置输入框回车发送
# def on_enter(widget):
#     on_send_button_clicked(None)
# input_box.on_submit(on_enter)

# # 显示主容器
# display(main_container)

In [7]:
def create_chat_interface():
    """创建聊天界面"""
    # 初始化模型和tokenizer
    model_path = "google/gemma-2-9b"
    cache_dir = "/root/autodl-tmp/gemma"
    # lora_path = "/root/autodl-tmp/models/stage1/gemma-base-zh-final"
    lora_path = None
    
    print("正在加载模型...")
    model, tokenizer = initialize_model_and_tokenizer(
        model_path=model_path,
        cache_dir=cache_dir,
        lora_path=lora_path,
        use_quantization=True
    )
    model.eval()
    print("模型加载完成!")

    # 修改输出区域的样式
    conversation_output = widgets.Output(
        layout=widgets.Layout(
            width='100%',
            max_width='1200px',
            min_height='400px',
            border='1px solid #ddd',
            overflow='auto',
            word_wrap='break-word'
        )
    )

    # 修改输入框的样式
    input_box = widgets.Text(
        value='',
        placeholder='请输入您的问题...',
        description='用户:',
        disabled=False,
        layout=widgets.Layout(
            width='80%',
            max_width='1000px'
        )
    )

    # 修改按钮样式
    send_button = widgets.Button(
        description='发送',
        disabled=False,
        button_style='primary',
        tooltip='发送消息',
        icon='paper-plane',
        layout=widgets.Layout(width='100px')
    )

    clear_button = widgets.Button(
        description='清空对话',
        button_style='warning',
        tooltip='清空对话历史',
        layout=widgets.Layout(width='100px')
    )

    # 按钮容器
    button_container = widgets.HBox(
        [send_button, clear_button],
        layout=widgets.Layout(
            width='auto',
            margin='0 0 0 10px'
        )
    )

    # 输入区域容器
    input_container = widgets.HBox(
        [input_box, button_container],
        layout=widgets.Layout(
            width='100%',
            max_width='1200px',
            justify_content='space-between'
        )
    )

    # 主容器
    main_container = widgets.VBox(
        [conversation_output, input_container],
        layout=widgets.Layout(
            width='100%',
            max_width='1200px',
            padding='10px',
            margin='0 auto'
        )
    )

    def on_send_button_clicked(b):
        user_input = input_box.value
        if not user_input.strip():
            return
            
        input_box.value = ''
        
        with conversation_output:
            # 显示用户输入
            print(f"\n用户: {user_input}\n")
            
            # 构建对话
            dialogue = [
                {
                    "role": "system",
                    "content": "你是一个专业、友好的AI助手。请用简洁、准确的方式回答问题。"
                },
                {
                    "role": "user",
                    "content": user_input
                }
            ]
            
            # 生成回答
            prompt = apply_chat_template(dialogue)
            response = generate_response(
                model,
                tokenizer,
                prompt,
                max_new_tokens=1024,
                temperature=0.7
            )
            
            print(f"助手: {response}\n")
            
    def on_clear_button_clicked(b):
        conversation_output.clear_output()

    # 绑定按钮事件
    send_button.on_click(on_send_button_clicked)
    clear_button.on_click(on_clear_button_clicked)
    
    # 显示界面
    display(main_container)

In [8]:
# 创建聊天界面
create_chat_interface()

正在加载模型...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

模型加载完成!
